# <u><center>Project 2 Part 5 Core
- Authored by: Eric N. Valdez
- Date: 4/6/2024

## `Sentiment Analysis and Rating Prediction of Moving Reviews`

### <u>Overview:
- This project is an extension of the movies porject from Enricment. This portion focuses on applying Natural Language Processing (NLP) techniques to analyze a database of movie reviews.
- Students will leverage NLP tools such as NLTK, Spacy, WordCloud, and Scikit-Learn to explore, analyze, and model text data. The ultimate goal is to establish a relationship between the textual content of the reviews and their associated ratings and subsequently predict these ratings.

### <u>Dataset: TMDB Movie Reviews
<center> <img src="Data-NLP/Images/IMDB.png">

#### [TMDB Movie Reviews](https://drive.google.com/file/d/1vLUzSYleJXqsjNMsq76yTQ5fmNlSHFJI/view). Ratings Range from 1 to 10</center>
- Gathering through `tmdbsimple` pyhton wrapper for the TMDB API. To legally cite TMDB, please follow their attribution requirements, which we have [summarized here](https://docs.google.com/document/d/1LzFQDulDdQjiMuZ8sBYeDbHnN62ZWjFU_xt_4eSwVIw/edit).

# <u>Imports:

In [1]:
import re
import matplotlib.pyplot as plt
import spacy
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import numpy as np
import nltk


from wordcloud import WordCloud
from wordcloud import STOPWORDS
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk import ngrams
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

# Increase column width
pd.set_option('display.max_colwidth', 250)
pd.set_option('display.max_colwidth',300)


# <u> Custom Functions:

In [2]:
def batch_preprocess_texts(
    texts,
    nlp=None,
    remove_stopwords=True,
    remove_punct=True,
    use_lemmas=False,
    disable=["ner"],
    batch_size=50,
    n_process=-1,
):
    """Efficiently preprocess a collection of texts using nlp.pipe()
    Args:
        texts (collection of strings): collection of texts to process (e.g. df['text'])
        nlp (spacy pipe), optional): Spacy nlp pipe. Defaults to None; if None, it creates a default 'en_core_web_sm' pipe.
        remove_stopwords (bool, optional): Controls stopword removal. Defaults to True.
        remove_punct (bool, optional): Controls punctuation removal. Defaults to True.
        use_lemmas (bool, optional): lemmatize tokens. Defaults to False.
        disable (list of strings, optional): named pipeline elements to disable. Defaults to ["ner"]: Used with nlp.pipe(disable=disable)
        batch_size (int, optional): Number of texts to process in a batch. Defaults to 50.
        n_process (int, optional): Number of CPU processors to use. Defaults to -1 (meaning all CPU cores).
    Returns:
        list of tokens
    """
    # from tqdm.notebook import tqdm
    from tqdm import tqdm
    if nlp is None:
        nlp = spacy.load("en_core_web_sm")
    processed_texts = []
    for doc in tqdm(nlp.pipe(texts, disable=disable, batch_size=batch_size, n_process=n_process)):
        tokens = []
        for token in doc:
            # Check if should remove stopwords and if token is stopword
            if (remove_stopwords == True) and (token.is_stop == True):
                # Continue the loop with the next token
                continue
            # Check if should remove stopwords and if token is stopword
            if (remove_punct == True) and (token.is_punct == True):
                continue
            # Check if should remove stopwords and if token is stopword
            if (remove_punct == True) and (token.is_space == True):
                continue
            
            ## Determine final form of output list of tokens/lemmas
            if use_lemmas:
                tokens.append(token.lemma_.lower())
            else:
                tokens.append(token.text.lower())
        processed_texts.append(tokens)
    return processed_texts

In [3]:
def classification_metrics(y_true, y_pred, label='',
                           output_dict=False, figsize=(8,4),
                           normalize='true', cmap='Blues',
                           colorbar=False):
  # Get the classification report
  report = classification_report(y_true, y_pred)
  ## Print header and report
  header = "-"*70
  print(header, f" Classification Metrics: {label}", header, sep='\n')
  print(report)
  ## CONFUSION MATRICES SUBPLOTS
  fig, axes = plt.subplots(ncols=2, figsize=figsize)
  # create a confusion matrix  of raw counts
  ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                normalize=None, cmap='gist_gray', colorbar=colorbar,
                ax = axes[0],);
  axes[0].set_title("Raw Counts")
  # create a confusion matrix with the test data
  ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                normalize=normalize, cmap=cmap, colorbar=colorbar,
                ax = axes[1]);
  axes[1].set_title("Normalized Confusion Matrix")
  # Adjust layout and show figure
  fig.tight_layout()
  plt.show()
  # Return dictionary of classification_report
  if output_dict==True:
    report_dict = classification_report(y_true, y_pred, output_dict=True)
    return report_dict

def evaluate_classification(model, X_train, y_train, X_test, y_test,
                         figsize=(6,4), normalize='true', output_dict = False,
                            cmap_train='Blues', cmap_test="Reds",colorbar=False):
  # Get predictions for training data
  y_train_pred = model.predict(X_train)
  # Call the helper function to obtain regression metrics for training data
  results_train = classification_metrics(y_train, y_train_pred, #verbose = verbose,
                                     output_dict=True, figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_train,
                                     label='Training Data')
  print()
  # Get predictions for test data
  y_test_pred = model.predict(X_test)
  # Call the helper function to obtain regression metrics for test data
  results_test = classification_metrics(y_test, y_test_pred, #verbose = verbose,
                                  output_dict=True,figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_test,
                                    label='Test Data' )
  if output_dict == True:
    # Store results in a dataframe if ouput_frame is True
    results_dict = {'train':results_train,
                    'test': results_test}
    return results_dict

In [4]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
def classification_metrics(y_true, y_pred, label='',
                           output_dict=False, figsize=(8,4),
                           normalize='true', cmap='Blues',
                           colorbar=False,values_format=".2f"):
    """Modified version of classification metrics function from Intro to Machine Learning.
    Updates:
    - Reversed raw counts confusion matrix cmap  (so darker==more).
    - Added arg for normalized confusion matrix values_format
    """
    # Get the classification report
    report = classification_report(y_true, y_pred)
    
    ## Print header and report
    header = "-"*70
    print(header, f" Classification Metrics: {label}", header, sep='\n')
    print(report)
    
    ## CONFUSION MATRICES SUBPLOTS
    fig, axes = plt.subplots(ncols=2, figsize=figsize)
    
    # Create a confusion matrix  of raw counts (left subplot)
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                                            normalize=None, 
                                            cmap='gist_gray_r',# Updated cmap
                                            values_format="d", 
                                            colorbar=colorbar,
                                            ax = axes[0]);
    axes[0].set_title("Raw Counts")
    
    # Create a confusion matrix with the data with normalize argument 
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                                            normalize=normalize,
                                            cmap=cmap, 
                                            values_format=values_format, #New arg
                                            colorbar=colorbar,
                                            ax = axes[1]);
    axes[1].set_title("Normalized Confusion Matrix")
    
    # Adjust layout and show figure
    fig.tight_layout()
    plt.show()
    
    # Return dictionary of classification_report
    if output_dict==True:
        report_dict = classification_report(y_true, y_pred, output_dict=True)
        return report_dict
    
    
def evaluate_classification(model, X_train, y_train, X_test, y_test,
                         figsize=(6,4), normalize='true', output_dict = False,
                            cmap_train='Blues', cmap_test="Reds",colorbar=False):
  # Get predictions for training data
  y_train_pred = model.predict(X_train)
  # Call the helper function to obtain regression metrics for training data
  results_train = classification_metrics(y_train, y_train_pred, #verbose = verbose,
                                     output_dict=True, figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_train,
                                     label='Training Data')
  print()
  # Get predictions for test data
  y_test_pred = model.predict(X_test)
  # Call the helper function to obtain regression metrics for test data
  results_test = classification_metrics(y_test, y_test_pred, #verbose = verbose,
                                  output_dict=True,figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_test,
                                    label='Test Data' )
  if output_dict == True:
    # Store results in a dataframe if ouput_frame is True
    results_dict = {'train':results_train,
                    'test': results_test}
    return results_dict

In [ ]:
# import matplotlib.pyplot as plt
# ax = dist.plot(20, show=False)
# ax.set_title('Number of Occurances of Top 20 Words')
# ax.grid(False)
# plt.tight_layout()
# plt.savefig('frequency_distribution.png')

In [14]:
def preprocess_doc(doc, remove_stopwords=True, remove_punct=True, use_lemmas=False):
    """Temporary Fucntion - for Education Purposes (we will make something better below)
    """
    tokens = [ ]
    for token in doc:
        # Check if should remove stopwords and if token is stopword
        if (remove_stopwords == True) and (token.is_stop == True):
            # Continue the loop with the next token
            continue
    
        # Check if should remove stopwords and if token is stopword
        if (remove_punct == True) and (token.is_punct == True):
            continue
    
        # Check if should remove stopwords and if token is stopword
        if (remove_punct == True) and (token.is_space == True):
            continue
    
        ## Determine final form of output list of tokens/lemmas
        if use_lemmas:
            tokens.append(token.lemma_.lower())
        else:
            tokens.append(token.text.lower())
    return tokens

# `Tasks`
## 0) <u>Update Your Project 2 Repo
- Create a new `Data-NLP/` folder in your project 2 repository
- Add the dowloaded review to this new `Data-NLP/` folder.
- Make sure you have an `` folder. If <u>not</u>, create one.

## 1) <u>Data Preprocessing 
- `Load and Inspect the dataset`
    - How many reviews?
    - What does the distribution of ratings look like?
    - Any Null values?
- `Use the rating column to create a new` <u>`target column`</u> `with 2 groups: high-rating and low-rating groups`
    - We recommend defining 'High-rating' reviews as any review with a rating >=9; and 'Low-Rating' reviews as any review with a rating <=4.
    - The middel ratings between 4 and 9 will be excluded from the analysis.
    - You may use an alternative definition for High & Low reviews, but justify your choice in your notebook/readme
- `Utilize NLTK & SpaCy for basic text processing including:`
    - Removing stopwards
    - Tokenization
    - Lemmatization
    - `TIPS:`
        - Be sure to creat a custom NLP Object & disable the named entity recognizer. Otherwise, processing will take a very long time!
        - **You will want to create several versions of the data, lemmatized, tolkenized, lemmatized joined back to one string per review, and tokenized joined back to one string per review.** This will be useful for different analysis and modeling techniques.
- `NOTE:`you may find some artifacts during your EDA e.g. HTML code like `"href"`. You are allowed to drop rows from the dataset after identifying problematic trends in some of the texts. `(Hint: Remember df[col].str.contains)`
- <u>Save your processed data frame in a `joblib` file saved in the `Data-NLP/` folder for future modeling. 

In [15]:
# Load the Data
mr = 'Data-NLP/movie_reviews_v2.csv'
text = pd.read_csv(mr, index_col='movie_id')
text.head()

,review_id,imdb_id,original_title,review,rating
movie_id,,,,,
843,64ecc16e83901800af821d50,tt0118694,花樣年華,"This is a fine piece of cinema from Wong Kar-Wai that tells us a story of two people whom circumstance throws together - but not in a way you might expect. We start with two couples who move into a new building. One a newspaper man with his wife, the other a business executive and his wife. The ...",7.0
7443,57086ff5c3a3681d29001512,tt0120630,Chicken Run,"A guilty pleasure for me personally, as I love both 'The Great Escape' and most of the works I have seen, over the years, from this rightfully-esteemed British animation company. Highly recommended both for children and for adults who enjoy animation.",9.0
7443,5bb5ac829251410dcb00810c,tt0120630,Chicken Run,"Made my roommate who hates stop-motion animation watched this in 2018 and even he had a good time. It's maybe not as great as I remember thinking it was when I was a little kid, but it still holds up to some degree.\r\n\r\n_Final rating:★★★ - I liked it. Would personally recommend you give it a ...",6.0
7443,5f0c53a013a32000357ec505,tt0120630,Chicken Run,"A very good stop-motion animation!\r\n\r\n<em>'Chicken Run'</em>, which I watched a crap tonne when I was little but not for a vast number of years now, is an impressive production given it came out in 2000. Despite a pretty simple feel to the film, it's a very well developed concept.\r\n\r\nThe...",8.0
7443,64ecc027594c9400ffe77c91,tt0120630,Chicken Run,"Ok, there is an huge temptation to riddle this review with puns - but I'm just going to say it's a cracking little family adventure. It's seemingly based on a whole range of classic movies from the ""Great Escape"", ""Star Trek"" to ""Love Story"" with a score cannibalised from just about any/everythi...",7.0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8650 entries, 0 to 8649
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   review_id       8650 non-null   object 
 1   movie_id        8650 non-null   int64  
 2   imdb_id         8650 non-null   object 
 3   original_title  8650 non-null   object 
 4   review          8650 non-null   object 
 5   rating          7454 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 405.6+ KB


Cleaing the data 

In [7]:
# Checking for duplicates
df.duplicated().sum()

0

In [8]:
# Checking for null Values
df.isna().sum()

review_id            0
movie_id             0
imdb_id              0
original_title       0
review               0
rating            1196
dtype: int64

In [22]:
# Dropping unnecesary columns
movie_reviews = df.drop(df.columns[[0, 1, 2]], axis=1)
movie_reviews.head()

,original_title,review,rating
0,花樣年華,"This is a fine piece of cinema from Wong Kar-Wai that tells us a story of two people whom circumstance throws together - but not in a way you might expect. We start with two couples who move into a new building. One a newspaper man with his wife, the other a business executive and his wife. The ...",7.0
1,Chicken Run,"A guilty pleasure for me personally, as I love both 'The Great Escape' and most of the works I have seen, over the years, from this rightfully-esteemed British animation company. Highly recommended both for children and for adults who enjoy animation.",9.0
2,Chicken Run,"Made my roommate who hates stop-motion animation watched this in 2018 and even he had a good time. It's maybe not as great as I remember thinking it was when I was a little kid, but it still holds up to some degree.\r\n\r\n_Final rating:★★★ - I liked it. Would personally recommend you give it a ...",6.0
3,Chicken Run,"A very good stop-motion animation!\r\n\r\n<em>'Chicken Run'</em>, which I watched a crap tonne when I was little but not for a vast number of years now, is an impressive production given it came out in 2000. Despite a pretty simple feel to the film, it's a very well developed concept.\r\n\r\nThe...",8.0
4,Chicken Run,"Ok, there is an huge temptation to riddle this review with puns - but I'm just going to say it's a cracking little family adventure. It's seemingly based on a whole range of classic movies from the ""Great Escape"", ""Star Trek"" to ""Love Story"" with a score cannibalised from just about any/everythi...",7.0


In [23]:
movie_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8650 entries, 0 to 8649
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  8650 non-null   object 
 1   review          8650 non-null   object 
 2   rating          7454 non-null   float64
dtypes: float64(1), object(2)
memory usage: 202.9+ KB


<u>Creating new columns:

In [29]:
# StopWords
from wordcloud import STOPWORDS
STOPWORDS

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'all',
 'also',
 'am',
 'an',
 'and',
 'any',
 'are',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 "can't",
 'cannot',
 'com',
 'could',
 "couldn't",
 'did',
 "didn't",
 'do',
 'does',
 "doesn't",
 'doing',
 "don't",
 'down',
 'during',
 'each',
 'else',
 'ever',
 'few',
 'for',
 'from',
 'further',
 'get',
 'had',
 "hadn't",
 'has',
 "hasn't",
 'have',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'hence',
 'her',
 'here',
 "here's",
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 "how's",
 'however',
 'http',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'k',
 "let's",
 'like',
 'me',
 'more',
 'most',
 "mustn't",
 'my',
 'myself',
 'no',
 'nor',
 'not',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'otherwise',
 'ought',
 'our',
 

In [ ]:
# Removing Stopwords


In [25]:
# Tolkenize
df['tokens'] = df['review'].map(lambda doc: doc.lower().split())
df.head()

,review_id,movie_id,imdb_id,original_title,review,rating,tokens
0,64ecc16e83901800af821d50,843,tt0118694,花樣年華,"This is a fine piece of cinema from Wong Kar-Wai that tells us a story of two people whom circumstance throws together - but not in a way you might expect. We start with two couples who move into a new building. One a newspaper man with his wife, the other a business executive and his wife. The ...",7.0,"[this, is, a, fine, piece, of, cinema, from, wong, kar-wai, that, tells, us, a, story, of, two, people, whom, circumstance, throws, together, -, but, not, in, a, way, you, might, expect., we, start, with, two, couples, who, move, into, a, new, building., one, a, newspaper, man, with, his, wife,,..."
1,57086ff5c3a3681d29001512,7443,tt0120630,Chicken Run,"A guilty pleasure for me personally, as I love both 'The Great Escape' and most of the works I have seen, over the years, from this rightfully-esteemed British animation company. Highly recommended both for children and for adults who enjoy animation.",9.0,"[a, guilty, pleasure, for, me, personally,, as, i, love, both, 'the, great, escape', and, most, of, the, works, i, have, seen,, over, the, years,, from, this, rightfully-esteemed, british, animation, company., highly, recommended, both, for, children, and, for, adults, who, enjoy, animation.]"
2,5bb5ac829251410dcb00810c,7443,tt0120630,Chicken Run,"Made my roommate who hates stop-motion animation watched this in 2018 and even he had a good time. It's maybe not as great as I remember thinking it was when I was a little kid, but it still holds up to some degree.\r\n\r\n_Final rating:★★★ - I liked it. Would personally recommend you give it a ...",6.0,"[made, my, roommate, who, hates, stop-motion, animation, watched, this, in, 2018, and, even, he, had, a, good, time., it's, maybe, not, as, great, as, i, remember, thinking, it, was, when, i, was, a, little, kid,, but, it, still, holds, up, to, some, degree., _final, rating:★★★, -, i, liked, it...."
3,5f0c53a013a32000357ec505,7443,tt0120630,Chicken Run,"A very good stop-motion animation!\r\n\r\n<em>'Chicken Run'</em>, which I watched a crap tonne when I was little but not for a vast number of years now, is an impressive production given it came out in 2000. Despite a pretty simple feel to the film, it's a very well developed concept.\r\n\r\nThe...",8.0,"[a, very, good, stop-motion, animation!, <em>'chicken, run'</em>,, which, i, watched, a, crap, tonne, when, i, was, little, but, not, for, a, vast, number, of, years, now,, is, an, impressive, production, given, it, came, out, in, 2000., despite, a, pretty, simple, feel, to, the, film,, it's, a,..."
4,64ecc027594c9400ffe77c91,7443,tt0120630,Chicken Run,"Ok, there is an huge temptation to riddle this review with puns - but I'm just going to say it's a cracking little family adventure. It's seemingly based on a whole range of classic movies from the ""Great Escape"", ""Star Trek"" to ""Love Story"" with a score cannibalised from just about any/everythi...",7.0,"[ok,, there, is, an, huge, temptation, to, riddle, this, review, with, puns, -, but, i'm, just, going, to, say, it's, a, cracking, little, family, adventure., it's, seemingly, based, on, a, whole, range, of, classic, movies, from, the, ""great, escape"",, ""star, trek"", to, ""love, story"", with, a, ..."


In [26]:
nlp_lite = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp_lite

In [28]:
# Lemmatization
df['lemmatized'] = batch_preprocess_texts(df['review'], nlp=nlp_lite,use_lemmas=True)
df.head()

1538it [00:34, 45.02it/s] 


KeyboardInterrupt: 

## <u>2) EDA and Visualizaton:
- `Create Word Clouds to visulize the most frequent and significant words in each group`
    - Remember, you can use this analysis to identify additional custom EDA stop words to use for visualization. (e.g. if the words are common in both groups)
    - <u>Save your WordClouds as .png files in the 'Images/' folder in your repo.</u>
- `Apply NLTK's` <u>`FreqDist`</u> `class to compare the frequency distribution of words in the review groups`
    - Remember, you can use this analysis to identify additional custom EDA stop words to use for visualization. (e.g. if the words are common in both groups)
    - <u>Save your `FreqDist` as .png files in the 'Images/' folder in your repo.</u>
- `Perform n-grams anaysis (bigrams and trigrams)`
    - Remember, you can use this analysis to identify additional custom stop words to use for EDA. (e.g., if the words are common in both groups)
    - Focus on BiGrams or TriGrams, using NLTK's `BigramCollectionFinder` and BigramAssocMeasures classes(or the Trigram equivalent Finder and Measures) to explore commonly used groups of words for each rating-group.
    - Describe any differences. What do these differences tell you?
    - <u>Save your data frame coparison of the top ngrams for each group as a Markdown Table.</u>
        - You can use the df.to_markdown() method to create a string version of your data frame that can be copied & pasted into a Markdown cell & your readme.
        - <center><img src='Images/1701388160__copymarkdowndataframe.png'></center>
- `Perform sentiment analysis to creat polarity scores according to VADER's sentiment lexicon`
    - Compare the sentiments of high-rating and low-rating texts
    - Compare the compound sentiment sores for high and low-rating reviews.
    - Which review polarity scores don't match the ratings? Why do you think this is?

In [ ]:
# from nltk.tokenize import word_tokenize
# # NLTK's Word Tokenization
# word_tokens = word_tokenize(sample_text.lower())
# print("Original text: \n", sample_text, '\n\n')
# print('Word tokens: \n', word_tokens)

In [ ]:
# from nltk.tokenize import TweetTokenizer
# # NLTK's Tweet Tokenization
# tweet_tokenizer = TweetTokenizer()
# tweet_tokens = tweet_tokenizer.tokenize(sample_text.lower())
# print("Original text: \n", sample_text, '\n\n')
# print(f"Tweet Tokens: \n", tweet_tokens,'\n')

In [ ]:
# from nltk import ngrams
# # Define bigrams
# bigrams = ngrams(tweet_tokens, 2)
# # display bigrams
# list(bigrams)

In [ ]:
# # define trigrams
# trigrams = ngrams(tweet_tokens, 3)
# # display trigrams
# list(trigrams)

## <u>3) Evaluation and Reporting:
- `Based on your anlysis, what should someone do (or not do) if they want to make a highly-rated movie?`
    - List 3 things associated with high-rating reviews
    - List 3 things associated with low-rating reviews 
- `Update your project README with a new Section for 'NLP Analysis of Movie Reviews.'`
    - Include what reviews were used (source and what the original rating numbers were b4 they were converted to a categorical target)
    - Include your EDA visualization in your README:
        - One WordCloud comparing both groups
        - 2 FreqDist plots(1 per group)
        - A Markdown table of the Top Ngrams for each group.
    - Your recommendations/conclusions for what to do/not to do make a highly-rated movie 

## <u>Deliverables:
1. Notebook files for Preprocessing and EDA
2. EDA Images saved in an `"Images"` folder.
3. Update README 